In [37]:
from uncertainties import ufloat
from uncertainties import unumpy as unp
import numpy as np

ImportError: cannot import name 'ucomplex' from 'uncertainties' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/uncertainties/__init__.py)

In [21]:
counts = np.random.rand(6,6)*1000
unc = np.random.rand(6,6)*100

data = unp.uarray(counts, unc)

In [28]:
def get_projections(raw_data) -> np.ndarray:
    # normalize groups of orthonormal measurements to get projections
    proj = np.zeros_like(raw_data)
    for i in range(0,6,2):
        for j in range(0,6,2):
            total_rate = np.sum(raw_data[i:i+2, j:j+2])
            proj[i:i+2, j:j+2] = raw_data[i:i+2, j:j+2]/total_rate
    
    # return the projections and uncertainties
    return proj

In [30]:
proj = get_projections(data)

In [33]:
def get_stokes(all_projs:np.ndarray) -> np.ndarray:
    # +++ oscar's code

    # unpack the projections
    HH, HV, HD, HA, HR, HL = all_projs[0]
    VH, VV, VD, VA, VR, VL = all_projs[1]
    DH, DV, DD, DA, DR, DL = all_projs[2]
    AH, AV, AD, AA, AR, AL = all_projs[3]
    RH, RV, RD, RA, RR, RL = all_projs[4]
    LH, LV, LD, LA, LR, LL = all_projs[5]

    # build the stokes's parameters
    S = np.zeros((4,4), dtype=object)
    S[0,0] = 1
    S[0,1] = DD - DA + AD - AA
    S[0,2] = RR + LR - RL - LL
    S[0,3] = HH - HV + VH - VV
    S[1,0] = DD + DA - AD - AA
    S[1,1] = DD - DA - AD + AA
    S[1,2] = DR - DL - AR + AL
    S[1,3] = DH - DV - AH + AV
    S[2,0] = RR - LR + RL - LL
    S[2,1] = RD - RA - LD + LA
    S[2,2] = RR - RL - LR + LL
    S[2,3] = RH - RV - LH + LV
    S[3,0] = HH + HV - VH - VV
    S[3,1] = HD - HA - VD + VA
    S[3,2] = HR - HL - VR + VL
    S[3,3] = HH - HV - VH + VV

    return S

In [34]:
S = get_stokes(proj)

In [50]:
def reconstruct_rho(stokes) -> np.ndarray:
    # define pauli matrices
    I = np.eye(2, dtype=complex)
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)
    P = [I, X, Y, Z]

    # compute rho uncertainty
    rho_real = np.zeros((4,4), dtype=object)
    rho_imag = np.zeros((4,4), dtype=object)
    
    for i1 in range(4):
        for i2 in range(4):
            rho_real += stokes[i1,i2] * np.array(np.real(np.kron(P[i1],P[i2])), dtype=object)
            rho_imag += stokes[i1,i2] * np.array(np.imag(np.kron(P[i1],P[i2])), dtype=object)

    # take the sqrt and divide by 4 to get the correct uncertainty
    rho_real /= 4
    rho_imag /= 4

    # return density matrix with uncertainties and the Stokes params unc for witness unc
    return rho_real, rho_imag

In [51]:
reconstruct_rho(S)

(array([[0.3894427878067723+/-0.026130917632550304,
         -0.07426704247743696+/-0.02202719301221163,
         0.11387965474746084+/-0.042473318704390084,
         -0.02625639291633053+/-0.029601580876726857],
        [-0.07426704247743696+/-0.02202719301221163,
         0.1628831451332889+/-0.00800706943332308,
         -0.30646109425783735+/-0.029601580876726857,
         -0.07536248342423155+/-0.042473318704390084],
        [0.11387965474746084+/-0.042473318704390084,
         -0.30646109425783735+/-0.029601580876726857,
         0.36104048958447693+/-0.016673550789818244,
         0.021723338806733824+/-0.02202719301221163],
        [-0.02625639291633053+/-0.029601580876726857,
         -0.07536248342423155+/-0.042473318704390084,
         0.021723338806733824+/-0.02202719301221163,
         0.0866335774754619+/-0.006222434085780625]], dtype=object),
 array([[0.0+/-0, -0.10911088691297235+/-0.01977139146040142,
         0.007629889172164131+/-0.019643682085763857,
         -0.02